# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [1]:
# Import initial libraries
import numpy as np
import pandas as pd
from citipy import citipy

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [2]:
# Create a set of random latitude and longitude combinations
n = 2000 # number of coordinates to generate. 10 to test. 2,000 for final submission.

lats = np.random.uniform(-90, 90, n)
lngs = np.random.uniform(-180, 180, n)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lats_lngs = zip(lats,lngs)       

In [3]:
# Add the latitudes and longitudes to a list
coordinates = list(lats_lngs)
coordinates

[(89.69360236090233, -53.61555945836544),
 (-35.85406485325704, -44.21592911178635),
 (54.100927463496646, -141.48490151814758),
 (-30.978860617628406, 49.14523313112679),
 (52.6407735939726, -48.89033768331788),
 (-41.88517129708138, 140.59916252740345),
 (-4.091364914839829, -157.88264481041816),
 (48.51748764905301, -102.31916453519536),
 (39.7349987142249, 160.2625122655076),
 (17.551202551879754, -98.35997389825717),
 (-1.5279547767542851, -1.1587328020953862),
 (32.50280315716651, -67.82731386134198),
 (31.1301729773854, -118.41669384226027),
 (-85.31483568382184, -165.92301317015276),
 (58.09051791203959, 141.48799085290295),
 (-33.4566138978071, 0.12965947148143186),
 (-59.434931836279695, 32.25792836481543),
 (-13.09831013861806, -74.08269874619575),
 (41.059083515377864, 38.27848258652634),
 (12.92079983131903, 33.928195177683335),
 (-23.529705585191323, 64.23941139211675),
 (6.7614988961483675, -13.053099462292835),
 (-7.103973588209783, -171.90588850777624),
 (67.2466204612

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [4]:
# Create an empty list for holding the cities
cities =[]

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city=citipy.nearest_city(coordinate[0], coordinate[1])

    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count

print(len(cities))

753


## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [5]:
# Import the requests library
import requests

# Import the time library
import time

# Import json
import json

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

In [6]:
# Assemble the OpenWeatherMap's API call
url="https://api.openweathermap.org/data/2.5/weather?&APPID="+weather_api_key

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [7]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
i=0
for city in cities:
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.city_name.replace(" ","+") + "," + city.country_code + '&units=imperial'
    print(city_url)
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city.city_name},{city.country_code}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
   
    city_weather = requests.get(city_url).json()
    # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description  

    if city_weather["cod"]==200:
        # success!
#         city_weather_formatted_str = json.dumps(city_weather, indent=4)
#         print(city_weather_formatted_str)

        # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description       
        long=round(city_weather['coord']['lon'],2)
        lat=round(city_weather['coord']['lat'],2)
        min_temp=city_weather['main']['temp_min'] # not always reported; not requested to be displayed but needed for Deliverable #2
        max_temp=city_weather['main']['temp_max']
        humidity=city_weather['main']['humidity']
        cloudiness=city_weather['clouds']['all']
        wind_speed=city_weather['wind']['speed']
        country=city_weather['sys']['country']
        city_name=city_weather['name']
        description=city_weather["weather"][0]["description"]
        city_datum={'Lat':lat,'Lng':long,'Min Temp':min_temp,'Max Temp':max_temp,'Humidity':humidity,'Cloudiness':cloudiness,'Wind Speed':wind_speed,'City':city_name,'Country':country,'Current Description':description}
#         print(city_datum)
        # Append the city information into the city_data list
        city_data.append(city_datum)
    else:
        print("HTML Error: "+city_weather["cod"]+" Message: "+city_weather["message"])
    i+=1
    

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=upernavik,gl&units=imperial
Processing Record 1 of Set 1 | upernavik,gl
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=cidreira,br&units=imperial
Processing Record 2 of Set 1 | cidreira,br
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=sitka,us&units=imperial
Processing Record 3 of Set 1 | sitka,us
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=taolanaro,mg&units=imperial
Processing Record 4 of Set 1 | taolanaro,mg
HTML Error: 404 Message: city not found
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=bonavista,ca&units=imperial
Processing Record 5 of Set 1 | bonavista,ca
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=portland,a

https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=arraial+do+cabo,br&units=imperial
Processing Record 50 of Set 1 | arraial do cabo,br
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=illoqqortoormiut,gl&units=imperial
Processing Record 1 of Set 2 | illoqqortoormiut,gl
HTML Error: 404 Message: city not found
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=hobart,au&units=imperial
Processing Record 2 of Set 2 | hobart,au
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=dikson,ru&units=imperial
Processing Record 3 of Set 2 | dikson,ru
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=byron+bay,au&units=imperial
Processing Record 4 of Set 2 | byron bay,au
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=puerto+baquerizo+moreno,ec&units=imperial
Pro

https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=vaini,to&units=imperial
Processing Record 50 of Set 2 | vaini,to
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=sentyabrskiy,ru&units=imperial
Processing Record 1 of Set 3 | sentyabrskiy,ru
HTML Error: 404 Message: city not found
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=kampot,kh&units=imperial
Processing Record 2 of Set 3 | kampot,kh
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=pevek,ru&units=imperial
Processing Record 3 of Set 3 | pevek,ru
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=madison+heights,us&units=imperial
Processing Record 4 of Set 3 | madison heights,us
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=roma,au&units=imperial
Processing Record 5 of Set 3 | roma,au
h

https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=chokurdakh,ru&units=imperial
Processing Record 49 of Set 3 | chokurdakh,ru
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=kisangani,cd&units=imperial
Processing Record 50 of Set 3 | kisangani,cd
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=barrow,us&units=imperial
Processing Record 1 of Set 4 | barrow,us
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=nushki,pk&units=imperial
Processing Record 2 of Set 4 | nushki,pk
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=touros,br&units=imperial
Processing Record 3 of Set 4 | touros,br
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=manuk+mangkaw,ph&units=imperial
Processing Record 4 of Set 4 | manuk mangkaw,ph
https://api.openweathermap.org/dat

HTML Error: 404 Message: city not found
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=bandarbeyla,so&units=imperial
Processing Record 48 of Set 4 | bandarbeyla,so
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=mendahara,id&units=imperial
Processing Record 49 of Set 4 | mendahara,id
HTML Error: 404 Message: city not found
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=vao,nc&units=imperial
Processing Record 50 of Set 4 | vao,nc
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=matara,lk&units=imperial
Processing Record 1 of Set 5 | matara,lk
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=kaberamaido,ug&units=imperial
Processing Record 2 of Set 5 | kaberamaido,ug
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=albanel,ca&units=imperial
P

...

https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=necocli,co&units=imperial
Processing Record 49 of Set 11 | necocli,co
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=seydi,tm&units=imperial
Processing Record 50 of Set 11 | seydi,tm
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=nola,cf&units=imperial
Processing Record 1 of Set 12 | nola,cf
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=bridgetown,ca&units=imperial
Processing Record 2 of Set 12 | bridgetown,ca
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=ampanihy,mg&units=imperial
Processing Record 3 of Set 12 | ampanihy,mg
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=olafsvik,is&units=imperial
Processing Record 4 of Set 12 | olafsvik,is
https://api.openweathermap.org/data/2.5/weat

https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=walvis+bay,na&units=imperial
Processing Record 49 of Set 12 | walvis bay,na
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=andenes,no&units=imperial
Processing Record 50 of Set 12 | andenes,no
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=oreanda,ua&units=imperial
Processing Record 1 of Set 13 | oreanda,ua
HTML Error: 404 Message: city not found
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=paide,ee&units=imperial
Processing Record 2 of Set 13 | paide,ee
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=ishigaki,jp&units=imperial
Processing Record 3 of Set 13 | ishigaki,jp
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=matamoros,mx&units=imperial
Processing Record 4 of Set 13 | matamoros

https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=alice+springs,au&units=imperial
Processing Record 49 of Set 13 | alice springs,au
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=thinadhoo,mv&units=imperial
Processing Record 50 of Set 13 | thinadhoo,mv
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=tucuman,ar&units=imperial
Processing Record 1 of Set 14 | tucuman,ar
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=lasa,cn&units=imperial
Processing Record 2 of Set 14 | lasa,cn
HTML Error: 404 Message: city not found
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=sabha,jo&units=imperial
Processing Record 3 of Set 14 | sabha,jo
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=sidi+ali,dz&units=imperial
Processing Record 4 of Set 14 | sidi ali

HTML Error: 404 Message: city not found
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=stornoway,gb&units=imperial
Processing Record 49 of Set 14 | stornoway,gb
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=dustlik,uz&units=imperial
Processing Record 50 of Set 14 | dustlik,uz
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=port+lincoln,au&units=imperial
Processing Record 1 of Set 15 | port lincoln,au
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=novopokrovka,ru&units=imperial
Processing Record 2 of Set 15 | novopokrovka,ru
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=cocobeach,ga&units=imperial
Processing Record 3 of Set 15 | cocobeach,ga
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=maningrida,au&units=imperial
Processing Recor

https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=petropavlovsk-kamchatskiy,ru&units=imperial
Processing Record 48 of Set 15 | petropavlovsk-kamchatskiy,ru
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=mozarlandia,br&units=imperial
Processing Record 49 of Set 15 | mozarlandia,br
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=geertruidenberg,nl&units=imperial
Processing Record 50 of Set 15 | geertruidenberg,nl
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=mishelevka,ru&units=imperial
Processing Record 1 of Set 16 | mishelevka,ru
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=ossora,ru&units=imperial
Processing Record 2 of Set 16 | ossora,ru
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=shelburne,ca&units=imperial
Processing Record 3 

In [8]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

676

## Add the weather data to a new DataFrame.

In [9]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [10]:
# Display sample data
city_data_df.head(10)

,Lat,Lng,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,City,Country,Current Description
0,72.79,-56.15,-5.46,-5.46,68,16,2.55,Upernavik,GL,few clouds
1,-30.18,-50.21,75.15,75.15,90,4,7.99,Cidreira,BR,clear sky
2,57.05,-135.33,24.96,24.96,66,100,9.17,Sitka,US,overcast clouds
3,48.65,-53.11,31.86,31.86,97,100,25.88,Bonavista,CA,snow
4,-38.33,141.60,64.78,64.78,59,25,8.34,Portland,AU,scattered clouds
5,-16.48,-151.75,83.03,83.03,74,80,18.05,Faanui,PF,broken clouds
6,49.23,-102.17,20.64,20.64,92,100,24.92,Oxbow,CA,light snow
7,50.68,156.12,31.93,31.93,85,100,2.59,Severo-Kuril'sk,RU,overcast clouds
8,4.88,-1.76,79.07,79.07,90,98,5.86,Takoradze,GH,overcast clouds
9,32.29,-64.78,55.44,55.44,62,40,9.22,Hamilton,BM,scattered clouds


In [16]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['Lat', 'Lng', 'Min Temp', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'City', 'Country', 'Current Description'],
      dtype='object')

In [22]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
col_names=["City", "Country", "Lat", "Lng", "Min Temp", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"]

# Recreate the DataFrame by using the new column order
city_data_df_unsorted=city_data_df[col_names]

# Sort the DataFrame by "Max Temp"
city_data_df_final = city_data_df_unsorted.sort_values("Max Temp", ascending=True)

# Display sample data
city_data_df_final.head(10)

,City,Country,Lat,Lng,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
93,Pevek,RU,69.70,170.31,-22.45,-22.45,100,86,3.31,overcast clouds
277,Norman Wells,CA,65.28,-126.83,-17.48,-17.48,99,27,1.88,scattered clouds
390,Aklavik,CA,68.22,-135.01,-17.23,-17.23,92,6,2.48,clear sky
437,Bilibino,RU,68.05,166.44,-16.42,-16.42,100,47,1.81,scattered clouds
50,Cherskiy,RU,68.75,161.30,-14.24,-14.24,96,11,6.13,few clouds
313,Leningradskiy,RU,69.38,178.42,-13.47,-13.47,100,100,3.56,overcast clouds
34,Tuktoyaktuk,CA,69.45,-133.04,-13.00,-13.00,77,20,9.22,few clouds
101,Saskylakh,RU,71.92,114.08,-12.89,-12.89,90,39,8.25,scattered clouds
184,Thompson,CA,55.74,-97.86,-8.28,-8.28,100,68,4.00,broken clouds
130,Chokurdakh,RU,70.63,147.92,-8.19,-8.19,85,84,8.25,broken clouds


In [23]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df_final.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Min Temp               float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [24]:
# Set the output file name
filename='WeatherPy_Database.csv'

# Export the city_data DataFrame into a CSV file
city_data_df_final.to_csv(filename)